<a href="https://colab.research.google.com/github/ptsurko/AIML-notebooks/blob/master/n_gram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from urllib.request import urlopen
from collections import OrderedDict
from random import random
import math

# http://computational-linguistics-class.org/assignment5.html
# https://github.com/kanikamohan/cswingers/blob/master/hw5/hw5_skeleton.py
# https://www.coursera.org/learn/language-processing/supplement/fdxeI/perplexity-computation


shakespeare_url = "http://norvig.com/ngrams/shakespeare.txt"

data = urlopen(shakespeare_url).readlines()
train_data = data[:-16]
# train_data = data[:4]
test_data = data[-15:]
train_data = map(lambda line: line.decode("utf-8"), train_data)
test_data = map(lambda line: line.decode("utf-8"), test_data)

dictionary = set()

words = []
for line in train_data: # files are iterable
    line = line[:-1] # remove \n
    words.extend(line.split())

    words.append('\n')

V = len(set(words))

print('V: ', V)
print('len(words): ', len(words))

test_words = []
for line in test_data: # files are iterable
    line = line[:-1] # remove \n
    test_words.extend(line.split())
    test_words.append('\n')

print('test_data: ', test_data)
print('len(test_words): ', len(test_words))

V:  33504
len(words):  1109602
test_data:  <map object at 0x7f3094e82b00>
len(test_words):  141


In [0]:
n = 5

def update_model(model, context, word):
  node = model
  words = context + [word]
  for w in words:
    if w not in node:
      node[w] = [{}, 0]

    node[w][1] += 1
    node = node[w][0]
  
def ngram(model, words, n=1):
  context = ['~'] * (n - 1)

  for word in words:
    # print('context: ', context)
    # print('word: ', word)
    update_model(model, context, word)

    if n >= 2:
      context.pop(0)
      context.append(word)
    
  return model

model = dict()
model = ngram(model, words, n)

In [0]:
def get_node(model, context):
  node = model
  weight = None
  for w in context:
    if w not in node:
      return None, 0
    
    weight = node[w][1]
    node = node[w][0]
  
  return node, weight


def random_text(model, n, length):
  context = ['~'] * (n - 1)
  text = ''
  for _ in range(length):
    next_words_dict, total_weight = get_node(model, context)
    prob_weight = random() * total_weight

    weight = 0
    for word in sorted(next_words_dict.keys()):
      weight += next_words_dict[word][1]
      if weight >= prob_weight:
        text += word + ' '
        
        if n >= 2:
          context.pop(0)
          context.append(word)
        break

  return text

text = random_text(model, n, 60)
print(text)

A MIDSUMMER-NIGHT'S DREAM 
 
 Now , signior , what news ? 
 
 Alcibiades banished ! 
 
 Banished , sir . 
 
 As well as Lewis of France , or the Earl of Warwick 
 The greatest man in England but the king . 
 
 Madam my interpreter , what says she ? 
 
 Bait 


In [0]:
node = get_node(model, ['in', 'his'])
print(node)

({'walks': [{',': [{'and': [{}, 1]}, 1]}, 1], 'eyes': [{';': [{'\n': [{}, 1]}, 1], ',': [{'distraction': [{}, 1]}, 1]}, 2], 'sleep': [{',': [{'\n': [{}, 1]}, 1], 'he': [{'does': [{}, 1]}, 1]}, 2], 'esteem': [{',': [{'\n': [{}, 1]}, 1]}, 1], 'service': [{'perishing': [{'.': [{}, 1]}, 1], ';': [{'you': [{}, 1]}, 1], '.': [{'\n': [{}, 1]}, 1]}, 3], 'profession': [{',': [{'and': [{}, 1]}, 1]}, 1], 'sphere': [{'.': [{'\n': [{}, 1]}, 1], ',': [{'\n': [{}, 1]}, 1]}, 2], 'pride': [{'or': [{'sharpness': [{}, 1]}, 1], '.': [{'\n': [{}, 2]}, 2]}, 3], 'virtue': [{',': [{'which': [{}, 1]}, 1]}, 1], 'power': [{'against': [{'you': [{}, 1]}, 1], 'to': [{'bring': [{}, 1]}, 1], '.': [{'You': [{}, 1]}, 1]}, 3], 'house': [{'\n': [{'From': [{}, 1], 'I': [{}, 1]}, 2]}, 2], 'idle': [{'fire': [{',': [{}, 1]}, 1]}, 1], 'proper': [{'stream': [{"o'erflows": [{}, 1]}, 1]}, 1], 'colour': [{':': [{'your': [{}, 1]}, 1]}, 1], 'will': [{'to': [{'give': [{}, 1]}, 1]}, 1], 'sides': [{'?': [{'is': [{}, 1]}, 1]}, 1], 'hea

In [0]:
def prob(model, context, word, V):
  node_prev, weight_prev = get_node(model, context)
  node, weight = get_node(model, context + [word])
  prob = (weight + 1) / (weight_prev + V)
  return prob

def perplexity(model, n, words):
  V = len(set(words))

  context = ['~'] * (n - 1)
  p = 1

  for word in words:
    p *= prob(model, context, word, V)

    if n >= 2:
      context.pop(0)
      context.append(word)
  total = p ** (-1/len(words))

  return total

print(test_words)
print('perplexity: ', perplexity(model, n, test_words))

['Some', 'loving', 'friends', 'convey', 'the', 'emperor', 'hence', ',', '\n', 'And', 'give', 'him', 'burial', 'in', 'his', "father's", 'grave', '.', '\n', 'My', 'father', 'and', 'Lavinia', 'shall', 'forthwith', '\n', 'Be', 'closed', 'in', 'our', "household's", 'monument', '.', '\n', 'As', 'for', 'that', 'heinous', 'tiger', ',', 'Tamora', ',', '\n', 'No', 'funeral', 'rite', ',', 'nor', 'man', 'in', 'mournful', 'weeds', ',', '\n', 'No', 'mournful', 'bell', 'shall', 'ring', 'her', 'burial', ';', '\n', 'But', 'throw', 'her', 'forth', 'to', 'beasts', 'and', 'birds', 'of', 'prey', '.', '\n', 'Her', 'life', 'was', 'beast-like', ',', 'and', 'devoid', 'of', 'pity', ';', '\n', 'And', ',', 'being', 'so', ',', 'shall', 'have', 'like', 'want', 'of', 'pity', '.', '\n', 'See', 'justice', 'done', 'on', 'Aaron', ',', 'that', "damn'd", 'Moor', ',', '\n', 'By', 'whom', 'our', 'heavy', 'haps', 'had', 'their', 'beginning', ':', '\n', 'Then', ',', 'afterwards', ',', 'to', 'order', 'well', 'the', 'state', ',

88.98651591262461